# Node2Vec cell similarity

Goal of this exploration is to find out, if [node2vec](https://snap.stanford.edu/node2vec/) algorith could be used to process single cell datasets. We will generate graph of `cell1 -> gene <- gene2` from 10x matrices and train node2vec from it.

In [1]:
import os
import scprep
download_path = "/datasets/singlecell/tut/"
import numpy as np
print(download_path)

/datasets/singlecell/tut/


Downloading dataset provided [here](https://data.mendeley.com/datasets/v6n743h5ng/1) and used in [this tutorial](https://www.krishnaswamylab.org/workshop)

In [ ]:
if not os.path.isdir(os.path.join(download_path, "scRNAseq", "T0_1A")):
    scprep.io.download.download_and_extract_zip(
        url="https://md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com/"
        "5739738f-d4dd-49f7-b8d1-5841abdbeb1e",
        destination=download_path)

Now, let's list and load data from all subdirs

In [3]:
import glob
import pandas as pd

dirs = glob.glob(download_path + "scRNAseq/*")
dfs = []
for d in dirs:
    dfs.append(scprep.io.load_10X(d, sparse=True, gene_labels='both'))

df = pd.concat(dfs)
del(dfs)

Generate list of cells and genes in format cell 1 - genes 1, 2, 6

In [55]:
cells_with_gene = df.values.nonzero()
cells_with_gene

(array([    0,     0,     0, ..., 31160, 31160, 31160]),
 array([   43,    45,    58, ..., 33661, 33662, 33664]))

Turn it into edgelist - pandas dataframe with columns like `source`,`target` and `weight` - in our case they'll correspond to `barcodes`, `gene` and `count`

In [56]:
barcodes = df.index
genes = df.columns
edgelist = pd.DataFrame().from_dict({"barcodes": cells_with_gene[0], "genes": cells_with_gene[1], "count": df.values[cells_with_gene]})
edgelist.head()

,barcodes,genes,count
0,0,43,1.0
1,0,45,1.0
2,0,58,1.0
3,0,59,1.0
4,0,60,1.0


In [8]:
edgelist = edgelist[edgelist['count'] > 10]  # remove lowest 2stddev of counts

Use [stellargraph](https://github.com/stellargraph/stellargraph) library to generate random walks for node2vec

In [9]:
import stellargraph as sg

Gs = sg.StellarGraph(edges=edgelist.rename(columns={"barcodes": "source", "genes": "target", "count": "weight"}))
Gs.info()

Generate random walks with vector length of 55, 10 walks for each cell and `p` and `q` set to incentivise breadth rather than depth in graph walks. This step is by far longest and could be optimized to use multicpu or even GPU

In [11]:
from stellargraph.data import BiasedRandomWalk

rw = BiasedRandomWalk(Gs)
walks = rw.run(nodes=edgelist["genes"].unique(), n=10, length=55, p=2, q=0.5)

In [57]:
print(len(walks))
print(walks[0])

17150
[197, 5971, 28546, 20215, 24922, 6044, 21625, 4766, 1933, 18034, 14381, 9037, 24655, 7200, 13864, 10884, 13367, 11054, 21042, 18740, 15689, 10856, 33480, 15984, 7908, 10314, 13434, 29259, 32176, 14296, 484, 19949, 26699, 20778, 31726, 25144, 5262, 29321, 33654, 17700, 5719, 3624, 16698, 30460, 21035, 26790, 11469, 6348, 30263, 23960, 14381, 17220, 21042, 10255, 7966]


Now, that we have walks, or "sencences" from our cells - let's train typical Word2Vec with them

In [13]:
from gensim.models import Word2Vec

weighted_model = Word2Vec(
    sentences=walks, vector_size=55, window=5, min_count=1, workers=4)


In [14]:
weighted_model.train(walks, total_examples=len(walks), epochs=3)

(2538358, 2829750)

To find most similar cells, we can use 

In [18]:
edgelist["genes"].unique()

array([  197,   484,   556, ...,  7246, 20807, 26907])

In [40]:
sims = weighted_model.wv.most_similar(7246, topn=10)
sims

[(17050, 0.999372661113739),
 (10213, 0.999346137046814),
 (10558, 0.9993376135826111),
 (5822, 0.9993347525596619),
 (6536, 0.999327540397644),
 (6373, 0.9993233680725098),
 (24147, 0.9993219375610352),
 (25204, 0.9993113279342651),
 (8930, 0.9992930293083191),
 (9187, 0.9992917776107788)]

In [50]:
cell_id = 7246

print(df.index[cell_id])
df.iloc[cell_id][df.iloc[cell_id].to_numpy().nonzero()[0]].sort_values(ascending=False).head(20)

AGTAGAGACATGAC-1


RPS2 (ENSG00000140988)      79.0
MT-CO3 (ENSG00000198938)    69.0
RPS18 (ENSG00000231500)     58.0
MT-CO1 (ENSG00000198804)    58.0
RPL13A (ENSG00000142541)    54.0
RPS3A (ENSG00000145425)     50.0
TUBA1B (ENSG00000123416)    50.0
RPS6 (ENSG00000137154)      50.0
RPL15 (ENSG00000174748)     49.0
RPLP1 (ENSG00000137818)     48.0
RPS19 (ENSG00000105372)     47.0
RPL41 (ENSG00000229117)     47.0
RPLP0 (ENSG00000089157)     45.0
EEF1A1 (ENSG00000156508)    44.0
RPL6 (ENSG00000089009)      43.0
RPS3 (ENSG00000149273)      41.0
RPL13 (ENSG00000167526)     40.0
RPL10 (ENSG00000147403)     37.0
RPS23 (ENSG00000186468)     36.0
RPL12 (ENSG00000197958)     35.0
Name: AGTAGAGACATGAC-1, dtype: Sparse[float64, 0.0]

In [51]:
cell_id = 17050

print(df.index[cell_id])
df.iloc[cell_id][df.iloc[cell_id].to_numpy().nonzero()[0]].sort_values(ascending=False).head(20)

TAGTCGGACGAGTT-1


RPS2 (ENSG00000140988)       72.0
RPL10 (ENSG00000147403)      63.0
RPL15 (ENSG00000174748)      62.0
RPL13A (ENSG00000142541)     62.0
RPL3 (ENSG00000100316)       60.0
GAPDH (ENSG00000111640)      56.0
RPS6 (ENSG00000137154)       53.0
RPLP0 (ENSG00000089157)      49.0
RPL13 (ENSG00000167526)      47.0
TUBA1B (ENSG00000123416)     43.0
RPS18 (ENSG00000231500)      42.0
EEF1A1 (ENSG00000156508)     37.0
RPS3 (ENSG00000149273)       36.0
RPS19 (ENSG00000105372)      34.0
RPS4X (ENSG00000198034)      33.0
MALAT1 (ENSG00000251562)     32.0
HNRNPA1 (ENSG00000135486)    32.0
RPL41 (ENSG00000229117)      31.0
RPL7 (ENSG00000147604)       31.0
RPL6 (ENSG00000089009)       31.0
Name: TAGTCGGACGAGTT-1, dtype: Sparse[float64, 0.0]